In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
from haversine import haversine

In [3]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')
    
#csv_to_parquet('input/train.csv', 'train')
#csv_to_parquet('input/test.csv', 'test')    

train Done.
test Done.


In [17]:
%%time
train = pd.read_csv('input/train.csv')

Wall time: 12.6 s


In [18]:
train06 = train[pd.to_datetime(train['base_date'], format='%Y%m%d').dt.strftime('%Y%m') == '202206']
train06.shape

(490137, 23)

In [19]:
train06.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490137 entries, 0 to 4701214
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     490137 non-null  object 
 1   base_date              490137 non-null  int64  
 2   day_of_week            490137 non-null  object 
 3   base_hour              490137 non-null  int64  
 4   lane_count             490137 non-null  int64  
 5   road_rating            490137 non-null  int64  
 6   road_name              490137 non-null  object 
 7   multi_linked           490137 non-null  int64  
 8   connect_code           490137 non-null  int64  
 9   maximum_speed_limit    490137 non-null  float64
 10  vehicle_restricted     490137 non-null  float64
 11  weight_restricted      490137 non-null  float64
 12  height_restricted      490137 non-null  float64
 13  road_type              490137 non-null  int64  
 14  start_node_name        490137 non-n

In [20]:
train06.to_parquet(f'./train06.parquet')

### data info 전환

In [ ]:
#data_info = pd.read_csv('input/data_info.csv')

In [ ]:
#data_info.to_excel('data_info.xlsx', encoding='utf-8-sig')

### https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36
weather = pd.read_csv('input/weather.csv', encoding='cp949')
weather['yyyymmdd'] = weather['일시'].str.slice(0,10).str.replace('-','').astype(int)
weather['hour'] = weather['일시'].str.slice(11,13).astype(int)
wehther = weather.fillna(0)
weather = weather.groupby(['yyyymmdd','hour']).mean()[['강수량(mm)']].reset_index()
weather.shape

#train = train.merge(weather, left_on=['base_date','base_hour'], right_on=['yyyymmdd','hour'], how='inner')
#test = test.merge(weather, left_on=['base_date','base_hour'], right_on=['yyyymmdd','hour'], how='inner')

## data load

In [ ]:
#!pip install pyarrow

In [2]:
train = pd.read_parquet('./train07.parquet') # 07
test = pd.read_parquet('./test.parquet')

In [11]:
import math
def truncate(number, digits) -> float:
    # Improve accuracy with floating point operations, to avoid truncate(16.4, 2) = 16.39 or truncate(-1.13, 2) = -1.12
    nbDecimals = len(str(number).split('.')[1]) 
    if nbDecimals <= digits:
        return number
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

In [43]:
num = 1
cols = ['start_latitude','start_longitude', 'end_latitude','end_longitude']

for c in cols:
    train[c] = list(map(lambda x: truncate(x,num), train[c]))

In [ ]:
cols 

In [44]:
grp = train.groupby(['base_date','base_hour'] + cols).agg({'target':['count','min', 'max', 'mean']})

In [45]:
grp.columns = ['t_cnt', 't_min', 't_max', 't_mean']

In [46]:
grp.sort_values('t_cnt')

t_cnt  \
base_date base_hour start_latitude start_longitude end_latitude end_longitude          
20220701  0         33.2           126.2           33.2         126.2              1   
20220709  15        33.2           126.5           33.3         126.6              1   
                                                   33.2         126.4              1   
                                   126.4           33.2         126.5              1   
20220726  3         33.3           126.6           33.2         126.5              1   
...                                                                              ...   
20220703  11        33.2           126.5           33.2         126.5            142   
20220721  10        33.2           126.5           33.2         126.5            142   
          12        33.2           126.5           33.2         126.5            142   
20220724  20        33.2           126.5           33.2         126.5            142   
20220701  0         33.2           126.5           33.2         126.5            143   

                                                                               t_min  \
base_date base_hour start_latitude start_longitude end_latitude end_longitude          
20220701  0         33.2           126.2           33.2         126.2           71.0   
20220709  15        33.2           126.5           33.3         126.6           57.0   
                                                   33.2         126.4           50.0   
                                   126.4           33.2         126.5           53.0   
20220726  3         33.3           126.6           33.2         126.5           53.0   
...                                                                              ...   
20220703  11        33.2           126.5           33.2         126.5            9.0   
20220721  10        33.2           126.5           33.2         126.5           11.0   
          12        33.2           126.5           33.2         126.5           13.0   
20220724  20        33.2           126.5           33.2         126.5            8.0   
20220701  0         33.2           126.5           33.2         126.5           13.0   

                                                                               t_max  \
base_date base_hour start_latitude start_longitude end_latitude end_longitude          
20220701  0         33.2           126.2           33.2         126.2           71.0   
20220709  15        33.2           126.5           33.3         126.6           57.0   
                                                   33.2         126.4           50.0   
                                   126.4           33.2         126.5           53.0   
20220726  3         33.3           126.6           33.2         126.5           53.0   
...                                                                              ...   
20220703  11        33.2           126.5           33.2         126.5           53.0   
20220721  10        33.2           126.5           33.2         126.5           55.0   
          12        33.2           126.5           33.2         126.5           55.0   
20220724  20        33.2           126.5           33.2         126.5           54.0   
20220701  0         33.2           126.5           33.2         126.5           60.0   

                                                                                  t_mean  
base_date base_hour start_latitude start_longitude end_latitude end_longitude             
20220701  0         33.2           126.2           33.2         126.2          71.000000  
20220709  15        33.2           126.5           33.3         126.6          57.000000  
                                                   33.2         126.4          50.000000  
                                   126.4           33.2         126.5          53.000000  
20220726  3         33.3           126.6           33.2         126.5          53.000000  
...    

In [ ]:
train[(train['base_date'] == 20220720) & (train['base_hour'] == 21) \
      & (train['start_latitude'] == 33.25) & (train['start_longitude'] == 126.60) \
      & (train['end_latitude'] == 33.29) & (train['end_longitude'] == 126.59)]

In [5]:
#train[(train['base_date'] == 20220701) & (train['base_hour'] == 0)].sort_values('target').to_excel('temp/20220701_00.xlsx', encoding='utf-8-sig') 

In [17]:
train = train.groupby('road_name').agg({'target':['min','max','mean', 'count']}).reset_index()#.sort_values('min')

In [13]:
train0701[:2]

road_name target                       
               min   max       mean count
0         -   16.0  74.0  50.328947    76
1       경찰로   30.0  33.0  31.785714    14

In [18]:
train.columns = ['road_name','t_min','t_max','t_mean','t_count']

In [22]:
train['sub'] = train['t_max'] - train['t_min']

In [29]:
train[train['sub'] <= 30.0].road_name.values

array(['동문로', '삼무로', '신광로', '연북로', '일반국도95호선', '일주동로', '임항로', '지방도1112호선',
       '지방도1116호선', '지방도1119호선'], dtype=object)

In [4]:
#train.sort_values('sub')

## EDA

In [ ]:
train.shape, test.shape

In [ ]:
train.info()

In [ ]:
train.isna().sum().sum(), test.isna().sum().sum()

In [ ]:
train.describe(include="all").T

In [ ]:
train[:3]

In [ ]:
# datetime
train['base_date'] = pd.to_datetime(train['base_date'], format='%Y%m%d')
test['base_date'] = pd.to_datetime(test['base_date'], format='%Y%m%d')

In [ ]:
train['base_date'].min(), train['base_date'].max()

In [ ]:
test['base_date'].min(), test['base_date'].max()

## 6월로 7월 예측

In [ ]:
train['base_date'] = pd.to_datetime(train['base_date'], format='%Y%m%d')
train07 = train[train['base_date'].dt.strftime('%Y%m') == '202207']
train07.shape

In [ ]:
test.shape

In [ ]:
train.groupby(['yyyymm']).agg({'target':['min','max','mean', 'count']})

In [ ]:
display(train07['start_latitude'].value_counts().sort_values())
display(test['start_latitude'].value_counts().sort_values())

In [ ]:
display(train07['start_latitude'].value_counts().sort_index())
display(test['start_latitude'].value_counts().sort_index())

In [ ]:
train07.columns

In [ ]:
# label encoding 
str_col = ['road_name']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train07[i])
    train07[i]=le.transform(train07[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [ ]:
train07[['weight_restricted','target']].corr()

In [ ]:
sns.pairplot(train07[['weight_restricted','target']])

In [ ]:
# one-hot
cols = ['day_of_week','connect_code','road_type', 'start_turn_restricted', 'end_turn_restricted', 'road_name']
train = pd.get_dummies(train, prefix_sep='_', sparse=False, drop_first=True, columns = cols)
test = pd.get_dummies(test, prefix_sep='_', sparse=False, drop_first=True, columns = cols)

In [ ]:
# del 
del_cols = ['height_restricted','vehicle_restricted', 'start_node_name', 'end_node_name']
train = train.drop(del_cols, axis=1)
test = test.drop(del_cols, axis=1)

In [ ]:
train['road_rating'].value_counts()

In [ ]:
train['multi_linked'].value_counts()

In [ ]:
display(train['connect_code'].value_counts())
display(test['connect_code'].value_counts())

In [ ]:
train['maximum_speed_limit'].value_counts()

In [ ]:
display(train['weight_restricted'].value_counts())
display(test['weight_restricted'].value_counts())

In [ ]:
train['height_restricted'].value_counts()
test['height_restricted'].value_counts()

In [ ]:
train['road_type'].value_counts()

In [ ]:
train['start_turn_restricted'].value_counts()

In [ ]:
# 위경도로 거리 도출
train["distance"] = train[["start_latitude", "start_longitude", "end_latitude", "end_longitude"]].apply(lambda x: haversine((x[0], x[1]), (x[2], x[3])), axis="columns")
test["distance"] = test[["start_latitude", "start_longitude", "end_latitude", "end_longitude"]].apply(lambda x: haversine((x[0], x[1]), (x[2], x[3])), axis="columns")

In [ ]:
train['road_name'].value_counts()

In [ ]:
train['start_node_name'].value_counts()

In [ ]:
train['vehicle_restricted'].value_counts()

## 데이터 전처리

In [ ]:
str_col = ['day_of_week','start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [ ]:
y_train = train['target'] 

X_train = train.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

test = test.drop(['id','base_date', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

print(X_train.shape)
print(y_train.shape)
print(test.shape)

## 모델링

In [ ]:
LR = lgb.LGBMRegressor(random_state=42).fit(X_train, y_train)

## 예측

In [ ]:
pred = LR.predict(test)

## 제출

In [ ]:
sample_submission = pd.read_csv('input/sample_submission.csv')

In [ ]:
sample_submission['target'] = pred
sample_submission.to_csv("output/submit.csv", index = False)